# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import  f1_score,precision_score,recall_score,accuracy_score,make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import TruncatedSVD
from sqlalchemy import create_engine
import pickle
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import trigrams
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from scipy.stats import randint as sp_randInt
from scipy.stats import uniform as sp_randFloat



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\magagunk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\magagunk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\magagunk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\magagunk\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('MessagesTables',engine)
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # Tokenize the text - separating the words
    words = word_tokenize(text)
    #removing stop words such as (is,the, if...)
    words = [w for w in words if w not in stopwords.words("english")]
    # Reduce words to their stems
    words = [PorterStemmer().stem(w) for w in words]
    # Reducing words to their root form
    words = [WordNetLemmatizer().lemmatize(w) for w in words]

    return words




### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
#building a machine pipeline with RandomForestClassifier as output
pipeline = Pipeline([('vect', TfidfVectorizer(tokenizer = tokenize, use_idf = True, smooth_idf = True, sublinear_tf = False)),
                     ('clf', MultiOutputClassifier(RandomForestClassifier()))
                    ])
            

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# Spliting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [6]:
#Training the pipeline
pipeline.fit(X_train, y_train)


C:\Users\magagunk\AppData\Local\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x000001A3F33208B0>)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_predict_test = pipeline.predict(X_test)
print(y_predict_test)

[[1 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


In [8]:

def Accuracy_F1 (model, X_test, y_test):
    '''
    Genrating f1 score, precision and recall 
    Input is model(which will be classification_repor) and X_test & y_test
    Output  Prints the Classification report
    '''
    y_pred = model.predict(X_test)
    for i, col in enumerate(y_test):
        print(col)
        print(classification_report(y_test[col], y_pred[:, i]))

In [9]:
Accuracy_F1(pipeline, X_test, y_test)

related
              precision    recall  f1-score   support

           0       0.70      0.41      0.52      1563
           1       0.83      0.94      0.88      4944
           2       0.42      0.30      0.35        47

    accuracy                           0.81      6554
   macro avg       0.65      0.55      0.58      6554
weighted avg       0.80      0.81      0.79      6554

request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5443
           1       0.85      0.49      0.62      1111

    accuracy                           0.90      6554
   macro avg       0.87      0.73      0.78      6554
weighted avg       0.89      0.90      0.89      6554

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      655

C:\Users\magagunk\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\magagunk\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\magagunk\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\U

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6257
           1       0.91      0.13      0.23       297

    accuracy                           0.96      6554
   macro avg       0.93      0.57      0.60      6554
weighted avg       0.96      0.96      0.95      6554

other_aid
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      5690
           1       0.52      0.03      0.06       864

    accuracy                           0.87      6554
   macro avg       0.69      0.51      0.50      6554
weighted avg       0.82      0.87      0.81      6554

infrastructure_related
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      6143
           1       0.50      0.00      0.00       411

    accuracy                           0.94      6554
   macro avg       0.72      0.50      0.49      6554
weighted avg       0.91      0.94      0.

In [10]:
#credit to siddhardhan from youtube on this helper code and some research on stackoverflow
# Below I am making a function for the f1_score, precision and recall to run through the data of each column

def Precision_recall_f1_score(true_value, pred_value):
    '''Making a function to put the Classification report in table format.
       Input actual values and predicted values
       Output is the classification report (shown as weighted, macro and micro)
    '''
    precision_weighted = precision_score(true_value, pred_value, average = "weighted" )
    recall_weighted = recall_score(true_value, pred_value, average = "weighted")
    f1_weighted = f1_score(true_value, pred_value, average = "weighted")
    precision_macro = precision_score(true_value, pred_value, average = "macro" )
    recall_macro = recall_score(true_value, pred_value, average = "macro")
    f1_macro = f1_score(true_value, pred_value, average = "macro")
    precision_micro = precision_score(true_value, pred_value, average = "micro" )
    recall_micro = recall_score(true_value, pred_value, average = "micro")
    f1_micro = f1_score(true_value, pred_value, average = "micro")
    return {"Precision_weighted": precision_weighted, "Recall_weighted": recall_weighted,"F1_Score_weighted": f1_weighted, 
            "Precision_macro": precision_macro, "Recall_macro": recall_macro,"F1_Score_macro": f1_macro, 
            "Precision_micro": precision_micro, "Recall_micro": recall_micro,"F1_Score_micro": f1_micro}


In [11]:
#making a new dataframe to run the f1_score, precision and recall function on each the columns

Test_Report = []
for m, column in enumerate(y_test.columns):
    Report = Precision_recall_f1_score(y_test.loc[:,column],y_predict_test[:,m])
    Test_Report.append(Report)
Test_Report_df = pd.DataFrame(Test_Report)


C:\Users\magagunk\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\magagunk\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\magagunk\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\magagunk\AppData\Local\anaconda3\l

In [12]:
Test_Report_df 

,Precision_weighted,Recall_weighted,F1_Score_weighted,Precision_macro,Recall_macro,F1_Score_macro,Precision_micro,Recall_micro,F1_Score_micro
0,0.797240,0.811108,0.792736,0.651652,0.550830,0.584140,0.811108,0.811108,0.811108
1,0.893805,0.897925,0.886248,0.874939,0.734022,0.779300,0.897925,0.897925,0.897925
2,0.989955,0.994965,0.992454,0.497482,0.500000,0.498738,0.994965,0.994965,0.994965
3,0.779050,0.780439,0.779272,0.773748,0.767985,0.770361,0.780439,0.780439,0.780439
4,0.920470,0.924474,0.895086,0.897647,0.543344,0.560026,0.924474,0.924474,0.924474
5,0.948712,0.951175,0.931281,0.925705,0.539003,0.559628,0.951175,0.951175,0.951175
6,0.966156,0.975893,0.965537,0.761035,0.518477,0.529188,0.975893,0.975893,0.975893
7,0.969479,0.981996,0.973671,0.616221,0.504077,0.503791,0.981996,0.981996,0.981996
8,0.960625,0.970247,0.958405,0.795212,0.531870,0.551258,0.970247,0.970247,0.970247
9,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
Test_Report_df.mean()

Precision_weighted    0.940028
Recall_weighted       0.948742
F1_Score_weighted     0.936964
Precision_macro       0.771370
Recall_macro          0.599206
F1_Score_macro        0.615966
Precision_micro       0.948742
Recall_micro          0.948742
F1_Score_micro        0.948742
dtype: float64

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
# adding GridSearchCV and tuning parameters
parameters = {'vect__max_df': (0.5, 1.0),
              'clf__estimator__n_estimators': [5, 10, 20],
              'clf__estimator__min_samples_split': [2]}

CV = GridSearchCV(pipeline, param_grid = parameters, cv = 3, refit = True, return_train_score = True,verbose=5)


In [15]:
CV.get_params()

{'cv': 3,
 'error_score': nan,
 'estimator__memory': None,
 'estimator__steps': [('vect',
   TfidfVectorizer(tokenizer=<function tokenize at 0x000001A3F33208B0>)),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'estimator__verbose': False,
 'estimator__vect': TfidfVectorizer(tokenizer=<function tokenize at 0x000001A3F33208B0>),
 'estimator__clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'estimator__vect__analyzer': 'word',
 'estimator__vect__binary': False,
 'estimator__vect__decode_error': 'strict',
 'estimator__vect__dtype': numpy.float64,
 'estimator__vect__encoding': 'utf-8',
 'estimator__vect__input': 'content',
 'estimator__vect__lowercase': True,
 'estimator__vect__max_df': 1.0,
 'estimator__vect__max_features': None,
 'estimator__vect__min_df': 1,
 'estimator__vect__ngram_range': (1, 1),
 'estimator__vect__norm': 'l2',
 'estimator__vect__preprocessor': None,
 'estimator__vect__smooth_idf': True,
 'estimator__vect__stop_words': None,
 

In [16]:
CV.fit(X_train, y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


C:\Users\magagunk\AppData\Local\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 1/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=5, vect__max_df=0.5;, score=(train=0.759, test=0.191) total time= 2.4min


C:\Users\magagunk\AppData\Local\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 2/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=5, vect__max_df=0.5;, score=(train=0.754, test=0.201) total time= 2.2min


...

[CV 2/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, vect__max_df=1.0;, score=(train=0.914, test=0.256) total time= 4.0min


C:\Users\magagunk\AppData\Local\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 3/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, vect__max_df=1.0;, score=(train=0.914, test=0.245) total time= 3.8min


C:\Users\magagunk\AppData\Local\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(tokenizer=<function tokenize at 0x000001A3F33208B0>)),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__min_samples_split': [2],
                         'clf__estimator__n_estimators': [5, 10, 20],
                         'vect__max_df': (0.5, 1.0)},
             return_train_score=True, verbose=5)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
#Predict against the refined model with GriSearchCV and RandomForestClassifier
version2_model = CV.best_estimator_
y_predict_test2 = version2_model.predict(X_test)

In [18]:
#Checking the outcome of the tuning in a dataframe format
Test_Report2 = []
for m, column in enumerate(y_test.columns):
    Report = Precision_recall_f1_score(y_test.loc[:,column], y_predict_test2[:, m])
    Test_Report2.append(Report)
Test_Report2_df = pd.DataFrame(Test_Report2)
Test_Report2_df 

C:\Users\magagunk\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\magagunk\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\magagunk\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\magagunk\AppData\Local\anaconda3\l

,Precision_weighted,Recall_weighted,F1_Score_weighted,Precision_macro,Recall_macro,F1_Score_macro,Precision_micro,Recall_micro,F1_Score_micro
0,0.787594,0.802716,0.787596,0.618326,0.557647,0.578661,0.802716,0.802716,0.802716
1,0.884794,0.890449,0.876803,0.860491,0.715193,0.759517,0.890449,0.890449,0.890449
2,0.989955,0.994965,0.992454,0.497482,0.500000,0.498738,0.994965,0.994965,0.994965
3,0.762519,0.764419,0.761614,0.758869,0.746800,0.750840,0.764419,0.764419,0.764419
4,0.907716,0.923406,0.896781,0.812541,0.552129,0.573755,0.923406,0.923406,0.923406
5,0.943027,0.950259,0.929992,0.875475,0.534401,0.551371,0.950259,0.950259,0.950259
6,0.963415,0.975587,0.964830,0.710381,0.512188,0.517627,0.975587,0.975587,0.975587
7,0.969479,0.981996,0.973671,0.616221,0.504077,0.503791,0.981996,0.981996,0.981996
8,0.959363,0.969942,0.960212,0.754628,0.553505,0.583631,0.969942,0.969942,0.969942
9,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [19]:
#Scores for each column
Test_Report2_df

,Precision_weighted,Recall_weighted,F1_Score_weighted,Precision_macro,Recall_macro,F1_Score_macro,Precision_micro,Recall_micro,F1_Score_micro
0,0.787594,0.802716,0.787596,0.618326,0.557647,0.578661,0.802716,0.802716,0.802716
1,0.884794,0.890449,0.876803,0.860491,0.715193,0.759517,0.890449,0.890449,0.890449
2,0.989955,0.994965,0.992454,0.497482,0.500000,0.498738,0.994965,0.994965,0.994965
3,0.762519,0.764419,0.761614,0.758869,0.746800,0.750840,0.764419,0.764419,0.764419
4,0.907716,0.923406,0.896781,0.812541,0.552129,0.573755,0.923406,0.923406,0.923406
5,0.943027,0.950259,0.929992,0.875475,0.534401,0.551371,0.950259,0.950259,0.950259
6,0.963415,0.975587,0.964830,0.710381,0.512188,0.517627,0.975587,0.975587,0.975587
7,0.969479,0.981996,0.973671,0.616221,0.504077,0.503791,0.981996,0.981996,0.981996
8,0.959363,0.969942,0.960212,0.754628,0.553505,0.583631,0.969942,0.969942,0.969942
9,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [20]:
#Summary of scores
Test_Report2_df.mean()

Precision_weighted    0.935808
Recall_weighted       0.946462
F1_Score_weighted     0.934592
Precision_macro       0.754927
Recall_macro          0.593718
F1_Score_macro        0.610428
Precision_micro       0.946462
Recall_micro          0.946462
F1_Score_micro        0.946462
dtype: float64

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [21]:
#building a machine pipeline with AdaBoostClassifier and adding new features as output
pipeline2 = Pipeline([('vect', CountVectorizer()), ('trunc', TruncatedSVD()), ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state = 42)))
])

In [22]:
#Tuning parameters and using them on GridSearchCV
parameters3 = {'clf__estimator__learning_rate': [0.0001, 0.001, 1.0, 1.0],
               'clf__estimator__n_estimators': [10, 50, 100, 500],
               'clf__estimator__algorithm': ["SAMME.R", 'SAMME']}

CV3 = GridSearchCV(pipeline2, param_grid = parameters3, cv = 3)

In [23]:
#fitting GridsearchCV on training data
CV3.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('trunc', TruncatedSVD()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier(random_state=42)))]),
             param_grid={'clf__estimator__algorithm': ['SAMME.R', 'SAMME'],
                         'clf__estimator__learning_rate': [0.0001, 0.001, 1.0,
                                                           1.0],
                         'clf__estimator__n_estimators': [10, 50, 100, 500]})

In [24]:
#Predict against the refined model with GriSearchCV and AdaBoostClassifier
version3_model = CV3.best_estimator_
y_predict_test3 = version3_model.predict(X_test)

In [25]:
#Dataframe to view the results from the AdaBoostClassifier
Test_Report3 = []
for m, column in enumerate(y_test.columns):
    Report1 = Precision_recall_f1_score(y_test.loc[:,column], y_predict_test3[:, m])
    Test_Report3.append(Report1)
Test_Report3_df = pd.DataFrame(Test_Report3)
Test_Report3_df 

C:\Users\magagunk\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\magagunk\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\magagunk\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\magagunk\AppData\Local\anaconda3\l

C:\Users\magagunk\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\magagunk\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\magagunk\AppData\Local\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\magagunk\AppData\Local\anaconda3\l

,Precision_weighted,Recall_weighted,F1_Score_weighted,Precision_macro,Recall_macro,F1_Score_macro,Precision_micro,Recall_micro,F1_Score_micro
0,0.660781,0.737107,0.674399,0.372534,0.348029,0.332952,0.737107,0.737107,0.737107
1,0.689706,0.830485,0.753577,0.415243,0.500000,0.453697,0.830485,0.830485,0.830485
2,0.989955,0.994965,0.992454,0.497482,0.500000,0.498738,0.994965,0.994965,0.994965
3,0.351193,0.592615,0.441027,0.296308,0.500000,0.372102,0.592615,0.592615,0.592615
4,0.843404,0.918370,0.879292,0.459185,0.500000,0.478724,0.918370,0.918370,0.918370
5,0.897781,0.947513,0.921977,0.473756,0.500000,0.486525,0.947513,0.947513,0.947513
6,0.952069,0.975740,0.963759,0.487870,0.500000,0.493861,0.975740,0.975740,0.975740
7,0.964915,0.982301,0.973530,0.491150,0.500000,0.495536,0.982301,0.982301,0.982301
8,0.939900,0.969484,0.954463,0.484742,0.500000,0.492253,0.969484,0.969484,0.969484
9,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [26]:
#Summary of the scores
Test_Report3_df.mean()

Precision_weighted    0.869763
Recall_weighted       0.926733
F1_Score_weighted     0.895238
Precision_macro       0.477366
Recall_macro          0.509667
F1_Score_macro        0.491142
Precision_micro       0.926733
Recall_micro          0.926733
F1_Score_micro        0.926733
dtype: float64

### 9. Export your model as a pickle file

In [27]:
#I will use the RandomForestClassifier with the GridsearchCV as it performed better than the AdaBoostClassifier
# Taking RandomForestClassifier with the GridsearchCV is a good opting as it prevents data leakage 

filename = 'CV.pkl'
pickle.dump(CV, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.